# Convolutional Neural Network

The objective of this file is to provide work for the implmentation and execution of the convolutional neural.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler


In [ ]:
option = 2

n_features = 16
n_outputs  = 5

if(option == 1):
    datapath = '../data/'
    filename = 'cho.txt'
else:
    datapath = '../data/'
    filename = 'iyer.txt'
    n_features = 12
    n_outputs  = 11


dataSet = np.genfromtxt(datapath+filename)

if(option != 1):
    print(len(dataSet))
    dataSet = np.delete(dataSet, np.where(dataSet == -1)[0], axis=0)
    print(len(dataSet))

X = dataSet [:,2:]

# Min-Max Normalization

#scaler = MinMaxScaler()

#X = scaler.fit_transform(X)

y = dataSet[:,1]


X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.67, 
                                                    random_state=42)

X = X.reshape((X.shape[0], X.shape[1],1))
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],1))




In [4]:
%load_ext tensorboard

In [5]:
import tensorflow as tf
import datetime, os

In [6]:
!rm -rf ./logs/

In [7]:

def create_model(filter_num=0, ker_size=0, dropout_r=0.0,dense=0):

    model = Sequential()
    
    model.add(Conv1D(filters=filter_num, kernel_size=ker_size, activation='relu', input_shape=(n_features, 1)))
    model.add(Dropout(dropout_r))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(dense, activation='relu'))
    model.add(Dense(n_outputs+1, activation='softmax'))
      
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [153]:
def train_model():
  filter_num = 64
  ker_num    = 3
  dropout_r    = 0.5
  dense      = 100

  model = create_model()
  
  model.summary()

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=X_train, 
            y=y_train, 
            epochs=100, 
            validation_data=(X_test, y_test), 
            callbacks=[tensorboard_callback])


ValueError: ker_num is not a legal parameter

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size = 10, verbose=0)

filter_num = [8, 16,32]
ker_size    = [2, 4, 6]
dropout_r  = [0.5,0.6,0.7]
dense      = [20,60,120]


param_grid = dict(filter_num=filter_num, ker_size=ker_size, dropout_r=dropout_r, dense=dense)

cv_method = KFold(n_splits=10, shuffle=True)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv_method, scoring='accuracy',verbose=10)
grid.fit(X,y)
print(grid.cv_results_)

In [10]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))


The best parameters are {'dense': 60, 'dropout_r': 0.6, 'filter_num': 16, 'ker_size': 4} with a score of 0.77


In [137]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 20604), started 1:18:33 ago. (Use '!kill 20604' to kill it.)

In [17]:
# Code for 2D-CNN

PIE_datapath = '../data/PIE_32x32/'
PIE_test_filenames = ['StTestFile'+str(i)+'.txt' for i in range(1,11)]
PIE_train_filenames = ['StTrainFile'+str(i)+'.txt' for i in range(1,11)]
print(PIE_test_filenames)
print(PIE_train_filenames)


PIE_train_filename = PIE_train_filenames[0]
PIE_train_data = np.genfromtxt(PIE_datapath+PIE_train_filename,delimiter=' ')
PIE_Xtrain = PIE_train_data[:,0:1024]
PIE_ytrain = PIE_train_data[:,1024]

PIE_test_filename = PIE_test_filenames[0]
PIE_test_data = np.genfromtxt(PIE_datapath+PIE_test_filename,delimiter=' ')
PIE_Xtest = PIE_test_data[:,0:1024]
PIE_ytest = PIE_test_data[:,1024]


['StTestFile1.txt', 'StTestFile2.txt', 'StTestFile3.txt', 'StTestFile4.txt', 'StTestFile5.txt', 'StTestFile6.txt', 'StTestFile7.txt', 'StTestFile8.txt', 'StTestFile9.txt', 'StTestFile10.txt']
['StTrainFile1.txt', 'StTrainFile2.txt', 'StTrainFile3.txt', 'StTrainFile4.txt', 'StTrainFile5.txt', 'StTrainFile6.txt', 'StTrainFile7.txt', 'StTrainFile8.txt', 'StTrainFile9.txt', 'StTrainFile10.txt']
[[0.13725  0.17647  0.17647  ... 0.035294 0.035294 0.035294]
 [0.05098  0.047059 0.043137 ... 0.26275  0.24314  0.22745 ]
 [0.047059 0.058824 0.078431 ... 0.035294 0.031373 0.027451]
 ...
 [0.56471  0.57647  0.57647  ... 0.16863  0.18039  0.19608 ]
 [0.023529 0.031373 0.039216 ... 0.019608 0.019608 0.019608]
 [0.023529 0.031373 0.043137 ... 0.019608 0.019608 0.019608]]
